In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [3]:
words = open('names.txt', 'r').read().splitlines()

In [4]:
len(words)

32033

In [8]:
words[:3]

['emma', 'olivia', 'ava']

In [7]:
chars = sorted(set(''.join(words)))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [115]:
block_size = 3
X, Y = [], []

context = [0] * block_size
for w in words:
    
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        
        context = context[1:] + [ix]
#         print(f'{context} -> {ix}' )
X = torch.tensor(X)
y = torch.tensor(Y)

In [51]:
X.shape, X.dtype

(torch.Size([32, 3]), torch.int64)

In [52]:
y.shape, y.dtype

(torch.Size([32]), torch.int64)

In [53]:
C = torch.randn(27,2)

In [54]:
emb = C[X]

In [60]:
W1 = torch.randn(6,100)
b1 = torch.randn(100)

In [57]:
torch.cat([emb[:,0,:],emb[:,1,:],emb[:,2,:]], dim = 1).shape

torch.Size([32, 6])

In [58]:
torch.cat(torch.unbind(emb,dim=1), dim=1).shape

torch.Size([32, 6])

In [63]:
h = torch.tanh(emb.view(32,6) @ W1 + b1)
h.shape

torch.Size([32, 100])

In [64]:
W2 = torch.randn(100,27)
b2 = torch.randn(27)

In [65]:
logits = h@W2 + b2

In [66]:
counts = logits.exp()

In [67]:
counts.shape

torch.Size([32, 27])

In [70]:
counts.sum(dim = 1, keepdims = True).shape

torch.Size([32, 1])

In [74]:
prob = counts/counts.sum(dim=1,keepdims = True)

In [78]:
prob.shape

torch.Size([32, 27])

In [79]:
torch.arange(32)

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])

In [83]:
[prob[i,j] for i,j in enumerate(Y)]

[tensor(4.2108e-08),
 tensor(7.3084e-12),
 tensor(3.0258e-14),
 tensor(8.7722e-15),
 tensor(6.4172e-09),
 tensor(1.6141e-06),
 tensor(0.6335),
 tensor(6.1603e-09),
 tensor(6.9956e-09),
 tensor(0.0007),
 tensor(7.8795e-08),
 tensor(6.9522e-11),
 tensor(1.5664e-10),
 tensor(1.3282e-06),
 tensor(5.7742e-07),
 tensor(2.3582e-13),
 tensor(3.6493e-05),
 tensor(2.1553e-11),
 tensor(2.8613e-08),
 tensor(5.7478e-10),
 tensor(1.3386e-12),
 tensor(0.5519),
 tensor(0.7823),
 tensor(4.1084e-14),
 tensor(1.9669e-11),
 tensor(0.0031),
 tensor(4.1127e-09),
 tensor(3.1611e-13),
 tensor(1.3195e-10),
 tensor(1.3185e-08),
 tensor(7.8726e-12),
 tensor(2.3157e-12)]

In [84]:
prob[torch.arange(32), Y]

tensor([4.2108e-08, 7.3084e-12, 3.0258e-14, 8.7722e-15, 6.4172e-09, 1.6141e-06,
        6.3349e-01, 6.1603e-09, 6.9956e-09, 7.4607e-04, 7.8795e-08, 6.9522e-11,
        1.5664e-10, 1.3282e-06, 5.7742e-07, 2.3582e-13, 3.6493e-05, 2.1553e-11,
        2.8613e-08, 5.7478e-10, 1.3386e-12, 5.5193e-01, 7.8235e-01, 4.1084e-14,
        1.9669e-11, 3.0591e-03, 4.1127e-09, 3.1611e-13, 1.3195e-10, 1.3185e-08,
        7.8726e-12, 2.3157e-12])

In [91]:
counts = logits.exp()
prob = counts/counts.sum(dim=1,keepdims = True)
loss = -prob[torch.arange(32), y].log().mean()
#------------------------------------------------------
loss_inbuilt = F.cross_entropy(logits, y)

In [92]:
loss - loss_inbuilt

tensor(0.)

In [116]:
W1 = torch.randn((6,100))
b1 = torch.randn(100)
W2 = torch.randn((100,27))
b2 = torch.randn(27)

In [117]:
parameters = [C,W1,b1,W2,b2]
for p in parameters:
    p.requires_grad = True

In [124]:
X.shape

torch.Size([228146, 3])

In [128]:
for _ in range(10):   
    batch = torch.randint(0, X.shape[0], (32,))
    emb = C[X[batch]]
    h = torch.tanh(emb.view(-1,6) @ W1 + b1)
    logits = h@W2 + b2
    loss = F.cross_entropy(logits, y[batch])
    print(loss.item())
    for p in parameters:
        p.grad = None
    loss.backward()
    for p in parameters:
        p.data -= 0.1 * p.grad

3.8609254360198975
5.466743469238281
4.792149066925049
5.273321151733398
4.272727012634277
5.869709491729736
6.226574897766113
5.083045959472656
4.1707282066345215
4.309988498687744
